## Read function utils

In [393]:
def readfile(textfile):
    file = open(textfile, 'r')
    text = file.readlines()
    file.close()
    return "\n".join(text)

## Read files from directory

In [ ]:
import os
textfiles = ['./dataset/' + filename for filename in os.listdir('./dataset')]
all_text = "\n".join([readfile(file) for file in textfiles])

In [394]:
# tokenize sentences and words
from nltk.tokenize import sent_tokenize, word_tokenize

all_tokenized_sentences=[]
for sent in sent_tokenize(all_text):
    all_tokenized_sentences.append([token for token in word_tokenize(sent)])

## Process sentence tokens

In [395]:
processed_sentences = [[sent.lower() for sent in tokenized_sent] for tokenized_sent in all_tokenized_sentences]
processed_sentences = [sent for sent in processed_sentences if len(sent) > 2] 

## Prepare Input and Target arrays

In [210]:
import numpy as np
from nltk.util import ngrams
word_input = np.array([words for sent in processed_sentences for words in ngrams(sent[:-1], 2)])
word_output = np.array([words for sent in processed_sentences for words in sent[2:]])

In [251]:
predictor={}
for a, b in zip(word_input, word_output):
    a0=a[0]
    a1=a[1]
    if predictor.get(a0, None) == None:
        predictor[a0]={a1:[b]}
    else:
        if predictor[a0].get(a1, None) == None:
            predictor[a0][a1]=[b]
        else:
            predictor[a0][a1].append(b)

In [252]:
from collections import Counter
for a0 in predictor:
    for a1 in predictor[a0]:
        predictor[a0][a1] = (len(predictor[a0][a1]), predictor[a0][a1])
    
    predictor[a0] = {k: v for k, v in sorted(predictor[a0].items(), key=lambda item: item[1], reverse=True)}
    
    for a1 in predictor[a0]:
        predictor[a0][a1] = predictor[a0][a1][1]
        predictor[a0][a1] = list({k: v for k, v in sorted(Counter(predictor[a0][a1]).items(), key=lambda item: item[1], reverse=True)}.keys())

## Dump python object

In [400]:
import pickle
with open("./objects/predictor.obj", 'wb') as pred_file:
    pickle.dump(predictor, pred_file)